## SVM
                        Before CV     After CV
       Training Score:  0.85026       0.88777
       Testing Score:   0.83898       0.88106

                precision    recall  f1-score   support

     CANDIDATE       0.85      0.65      0.73       523
     CONFIRMED       0.75      0.87      0.81       594
     FALSE POS       0.98      1.00      0.99      1069

     micro avg       0.88      0.88      0.88      2186
     macro avg       0.86      0.84      0.84      2186
     weighted avg    0.88      0.88      0.88      2186

## Random Forest
                        Before CV     After CV
       Training Score:  0.99497       1.0
       Testing Score:   0.87740       0.89616

  
                precision    recall  f1-score   support

     CANDIDATE       0.84      0.73      0.78       523
     CONFIRMED       0.81      0.86      0.83       594
     FALSE POS       0.97      1.00      0.98      1069

     micro avg       0.90      0.90      0.90      2186
     macro avg       0.87      0.86      0.87      2186
     weighted avg    0.89      0.90      0.89      2186

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=e6281700d6ba4ed66ac65e14a084b8e73a5f85f47fb41fe891c6fa78cdc2c42e
  Stored in directory: /Users/claudedavid/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
# df.head()
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8.744000e+03,8.744000e+03,8744.000000,8744.000000,8744.000000,...,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000
mean,0.157136,0.241194,0.201167,0.125114,56.080618,1.839523e-03,-1.839523e-03,164.563271,0.009300,-0.009300,...,-161.354758,4.310223,0.120733,-0.140411,1.704566,0.352884,-0.379933,292.075061,43.828259,14.273969
std,0.363950,0.427832,0.400895,0.330867,117.385280,7.276504e-03,7.276504e-03,66.476457,0.021662,0.021662,...,72.986448,0.431557,0.132813,0.082936,5.682429,0.906364,1.810943,4.772918,3.599786,1.343509
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,...,-1762.000000,0.047000,0.000000,-1.207000,0.109000,0.000000,-103.825000,279.852720,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.667824,5.286750e-06,-2.456250e-04,132.729408,0.001200,-0.010000,...,-197.000000,4.215750,0.043000,-0.195000,0.826000,0.125750,-0.247000,288.670237,40.805911,13.474000
50%,0.000000,0.000000,0.000000,0.000000,8.970985,3.323000e-05,-3.323000e-05,136.910235,0.004020,-0.004020,...,-159.000000,4.439000,0.070000,-0.127000,0.997000,0.246000,-0.111000,292.285005,43.703989,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.190033,2.456250e-04,-5.286750e-06,169.975942,0.010000,-0.001200,...,-112.000000,4.544000,0.149000,-0.087000,1.346250,0.356000,-0.069000,295.900510,46.722135,15.318250
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,...,0.000000,5.364000,1.472000,0.000000,180.013000,33.091000,0.000000,301.720760,52.336010,19.065000


# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
target = df["koi_disposition"]
data = df.drop("koi_disposition", axis=1)
feature_names = data.columns
data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
8017,0,1,1,0,0.806277,4.947000e-06,-4.947000e-06,131.785670,0.006720,-0.006720,...,-184.0,4.471,0.054,-0.229,0.996,0.324,-0.108,290.81723,38.539120,13.614
1233,0,1,1,0,3.582077,4.318000e-06,-4.318000e-06,355.515064,0.000864,-0.000864,...,-235.0,4.422,0.090,-0.195,0.993,0.283,-0.131,296.07822,43.136940,15.193
2592,0,0,0,0,5.060923,2.616000e-05,-2.616000e-05,134.473160,0.004730,-0.004730,...,-112.0,4.492,0.048,-0.112,0.911,0.121,-0.060,289.91742,40.828606,13.346
4770,0,1,0,1,8.480304,3.320000e-07,-3.320000e-07,135.854534,0.000031,-0.000031,...,-169.0,3.946,0.195,-0.105,2.210,0.375,-0.563,298.80020,46.665539,7.631
6632,0,0,0,1,4.994716,4.495000e-05,-4.495000e-05,136.183300,0.009500,-0.009500,...,-194.0,3.706,0.320,-0.080,2.830,0.458,-1.068,282.58215,46.815510,13.352


# Pre-processing

Scale the data using the MinMaxScaler

In [8]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
# y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
# y_train_minmax = y_minmax.transform(y_train)
# y_test_minmax = y_minmax.transform(y_test)

# Train the Support Vector Machine

In [9]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_minmax, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [10]:
print(f"Training Data Score: {model.score(X_train_minmax, y_train)}")
print(f"Testing Data Score: {model.score(X_test_minmax, y_test)}")
# print('Test Acc: %.3f' % model.score(X_test, y_test))

Training Data Score: 0.8502592253735896
Testing Data Score: 0.838975297346752


# Hyperparameter Tuning

Use `GridSearchCV` to tune the `C` and `gamma` parameters

In [11]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [12]:
# Train the model with GridSearch
grid.fit(X_train_minmax, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.840, total=   1.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.857, total=   1.2s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.5s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.825, total=   1.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.856, total=   1.1s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.850, total=   1.1s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.840, total=   0.9s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.857, total=   1.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.825, total=   0.7s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.856, total=   0.7s
[CV] C=1, gamma=0.0005 ...............................................
[CV] .

[CV] .................. C=50, gamma=0.0001, score=0.880, total=   1.0s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.886, total=   1.1s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.865, total=   0.9s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.890, total=   1.3s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.902, total=   1.0s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .................. C=50, gamma=0.0005, score=0.880, total=   1.0s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .................. C=50, gamma=0.0005, score=0.886, total=   1.1s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  1.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.8845719149411174


In [14]:
grid.score(X_train_minmax, y_train)

0.8877706617871303

In [15]:
grid.score(X_test_minmax, y_test)

0.8810612991765783

In [16]:
predictions = grid.predict(X_test_minmax)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.85      0.65      0.73       523
     CONFIRMED       0.75      0.87      0.81       594
FALSE POSITIVE       0.98      1.00      0.99      1069

      accuracy                           0.88      2186
     macro avg       0.86      0.84      0.84      2186
  weighted avg       0.88      0.88      0.88      2186



In [18]:
# save fitted model to file
import joblib
filename = 'svm.sav'
joblib.dump(grid, filename)

['svm.sav']

# Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf = rf.fit(X_train_minmax, y_train)
print(rf.score(X_train_minmax, y_train))
print(rf.score(X_test_minmax, y_test))

1.0
0.8938700823421775


In [20]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.11767643797479063, 'koi_fpflag_co'),
 (0.10255828262138962, 'koi_fpflag_nt'),
 (0.0643387980041693, 'koi_fpflag_ss'),
 (0.05772228150001465, 'koi_model_snr'),
 (0.0430959644904659, 'koi_prad'),
 (0.038115781690686794, 'koi_duration_err1'),
 (0.03607299665512791, 'koi_fpflag_ec'),
 (0.03570225458167991, 'koi_prad_err1'),
 (0.0337895245775445, 'koi_duration_err2'),
 (0.030321675048767798, 'koi_prad_err2'),
 (0.02997146384304293, 'koi_steff_err1'),
 (0.027762649668695114, 'koi_steff_err2'),
 (0.025023318385774294, 'koi_duration'),
 (0.022700162915702107, 'koi_period'),
 (0.022172888848186863, 'koi_time0bk_err2'),
 (0.021678320113581593, 'koi_time0bk_err1'),
 (0.021419794340521863, 'koi_depth'),
 (0.019899388946279482, 'koi_impact'),
 (0.018627046760216766, 'koi_period_err2'),
 (0.018377569581245798, 'koi_insol_err1'),
 (0.017622353249047554, 'koi_period_err1'),
 (0.013890269723578406, 'koi_teq'),
 (0.013827677193206017, 'koi_time0bk'),
 (0.013086800351315637, 'ra'),
 (0.01261708753494

In [21]:
param_grid2 = {'n_estimators': [250, 300, 350],
              'max_depth': [125, 150, 175]}
grid2 = GridSearchCV(rf, param_grid2, verbose=3)

In [22]:
grid2.fit(X_train_minmax, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... max_depth=125, n_estimators=250, score=0.902, total=   8.4s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.4s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.898, total=   7.3s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.7s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.876, total=   6.8s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.892, total=   6.8s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.911, total=   6.7s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.904, total=   8.2s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.896, total=   8.2s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.878, total=   9.5s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.889, total=   9.5s
[CV] max_depth=125, n_estimators=300 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  6.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [23]:
print(grid2.best_params_)
print(grid2.best_score_)

{'max_depth': 125, 'n_estimators': 250}
0.8955491525739057


In [24]:
grid2.score(X_train_minmax, y_train)

1.0

In [25]:
grid2.score(X_test_minmax, y_test)

0.8966148215919487

In [26]:
predictions2 = grid2.predict(X_test_minmax)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions2))

                precision    recall  f1-score   support

     CANDIDATE       0.84      0.74      0.78       523
     CONFIRMED       0.81      0.85      0.83       594
FALSE POSITIVE       0.97      1.00      0.98      1069

      accuracy                           0.90      2186
     macro avg       0.87      0.86      0.87      2186
  weighted avg       0.90      0.90      0.89      2186



In [28]:
# save fitted model to file
import joblib
filename = 'random_forest.sav'
joblib.dump(grid, filename)

['random_forest.sav']